In [1]:
import pandas as pd

In [2]:
file = 'Prototyping LCI_for standard format.xlsx'

xl = pd.ExcelFile(file)
res = len(xl.sheet_names)

In [3]:
res

3

In [4]:
xl.sheet_names

['Feedstock harvest', 'Preprocessing', 'Fuel Production']

In [5]:
import pandas as pd
# from lookup_table import lookup_table
from utils import unit_conversion, convert_transport_lci, step_processing, used_other_process, process, format_input, calculate_lca, lookup_table

category = pd.read_csv("category.csv", index_col=0, header=0).squeeze()

def calc(file):
#     lci_feedstock = pd.read_excel(lci_file, sheet_name='Feedstock harvest')
#     lci_feedstock = format_input(lci_feedstock)

#     lci_preprocessing = pd.read_excel(lci_file, sheet_name='Preprocessing')
#     lci_preprocessing = format_input(lci_preprocessing)

#     lci_fuel = pd.read_excel(lci_file, sheet_name='Fuel Production')
#     lci_fuel = format_input(lci_fuel)
    
    xl = pd.ExcelFile(file)
    sheet_names = xl.sheet_names
    step_mapping = dict()
    for i, sheet in enumerate(sheet_names):
        df = pd.read_excel(file, sheet_name=sheet)
        df = format_input(df)
        step_mapping.update({sheet.lower(): df})
    # step_mapping = {'feedstock harvest': lci_feedstock, 'preprocessing': lci_preprocessing, 'fuel production': lci_fuel}
    # return step_mapping
    res = process(step_mapping)

    overall_lci = res[sheet_names[-1].lower()]
    overall_lci = overall_lci[overall_lci['Type']=='Input'].copy()
    # overall_lci['ID'] = overall_lci.apply(
    #     # lambda a: a['Resource'] if (pd.isna(a['End Use']))|(a['Resource'] == 'Electricity') else a['Resource']+'_'+a['End Use'], axis=1
    #     lambda a: a['Resource'] if pd.isna(a['End Use']) else a['Resource']+'_'+a['End Use'], axis=1
    # )
    overall_lci["End Use"] = overall_lci["End Use"].fillna("")
    overall_lci["ID"] = overall_lci.apply(
        # lambda a: a['Resource'] if (pd.isna(a['End Use']))|(a['Resource'] == 'Electricity') else a['Resource']+'_'+a['End Use'], axis=1
        # lambda a: a['Resource'] if pd.isna(a['End Use']) else a['Resource']+'_'+a['End Use'], axis=1
        lambda a: a["Resource"]
        if a["End Use"] == ""
        else a["Resource"] + "_" + a["End Use"],
        axis=1,
    )

    res = calculate_lca(overall_lci)
    res.loc[res['Category']!='Co-Product', 'Category'] = res.loc[res['Category']!='Co-Product', 'Resource'].map(category)
    res['Resource'] = res['Resource'].str.title()
    res['Resource'] = res['Resource'].str.replace('Wwt', 'WWT').str.replace('Fgd', 'FGD')

    return res

In [6]:
calc(file)

,Type,Process,Category,Resource,End Use,Input Amount,Unit,LHV,Density,Note,...,PM2.5_Sum,SOx_Sum,BC_Sum,OC_Sum,CH4_Sum,N2O_Sum,CO2_Sum,Biogenic CO2_Sum,CO2 (w/ C in VOC & CO)_Sum,GHG_Sum
0,Input,Fuel Production,Chemical,Caustic,,3902.541936,g,NaN,NaN,NaN,...,0.464485,3.722279,0.048252,0.144748,18.804377,0.171587,7221.114013,0.0,7231.218679,7840.820647
1,Input,Fuel Production,Chemical,Ammonia,,4134.032212,g,NaN,NaN,NaN,...,0.360105,3.060091,0.060211,0.145334,30.308742,0.203912,9831.453658,0.0,9932.147909,10895.446774
2,Input,Fuel Production,Chemical,Corn Steep Liquor,,2070.439280,g,NaN,NaN,NaN,...,0.381768,2.234203,0.072736,0.064678,4.195803,5.587219,1845.409314,0.0,1855.437542,3461.924721
3,Input,Fuel Production,Chemical,Diammonium Phosphate,,232.216060,g,NaN,NaN,NaN,...,0.193836,2.565899,0.011172,0.024666,0.880303,0.008202,357.702378,0.0,359.517656,388.100359
4,Input,Fuel Production,Chemical,Corn Oil,,16.336492,g,NaN,NaN,NaN,...,0.000057,0.000513,0.000004,0.000017,0.001993,0.000017,0.995630,0.0,0.997157,1.061544
5,Input,Fuel Production,Chemical,Flocculant,,982.064692,g,NaN,NaN,NaN,...,0.070317,0.495777,0.007510,0.024075,3.086775,0.029476,1147.193183,0.0,1148.800692,1249.215020
6,Input,Fuel Production,Chemical,Glucose,,2957.136030,g,NaN,NaN,NaN,...,0.204597,1.116202,0.037481,0.044519,4.052458,2.383274,1522.224163,0.0,1527.866373,2281.007764
7,Input,Fuel Production,Chemical,Host Nutrients,,82.370643,g,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
8,Input,Fuel Production,Chemical,Sulfur Dioxide,,20.075911,g,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
9,Input,Fuel Production,Chemical,Hydrogen,,1902.818979,g,119.986744,0.090052,NaN,...,0.586761,2.945234,0.072504,0.160952,40.909136,0.127386,16945.648661,0.0,16958.040846,18219.072195


In [17]:
for key, value in t.items():
    print(key)
    print(value)

Feedstock harvest
     Type               Process        Category           Resource  \
0   Input  Feedstock production         Farming  corn stover_1 leg   
1   Input  Feedstock production            Fuel             diesel   
2   Input  Feedstock production            Fuel             diesel   
3   Input  Feedstock production     Electricity        electricity   
5  Output                   NaN    Main product        corn stover   
0   Input  Feedstock production  Transportation             diesel   
1   Input  Feedstock production  Transportation             diesel   

                           End Use        Amount   Unit Note        LHV  \
0                              NaN      1.096491    ton  NaN        NaN   
1                  farming tractor  97631.578947    btu  NaN  42.626181   
2  stationary reciprocating engine      0.000000    btu  NaN  42.626181   
3                         u.s. mix     33.991228    btu  NaN        NaN   
5                              NaN      1.0000